# Chatbot

In [5]:
import os
from openai import OpenAI
import gradio as gr

In [7]:
#setup 
ollama_url = 'http://localhost:11434/v1'


ollama = OpenAI(base_url=ollama_url, api_key="ollama")


system_prompt = "You are a helpful assistant"

write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

With Gradio now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

I will write a function `chat(message, history)` where:  
- **message** is the prompt to use  
- **history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [10]:
#Chat function
def chat(message, history):
    # create a list for messages containing the system promt, history, and user promt
    messages = [{'role':'system', 'content': system_prompt}] + history + [{'role':'user', 'content':message}]

    print("History is:")
    print(history)
    print("Messages is:")
    print(messages)

    stream = ollama.chat.completions.create(model='llama3.1', messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [15]:
# Create Gradio interface
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


## Multi-Shot prompting and Context Enrichment

In [16]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [18]:
def chat(message, history):
    messages = [{'role':'system', 'content': system_message}] + history + [{'role':'user', 'content':message}]

    stream = ollama.chat.completions.create(model='llama3.1', messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


In [24]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are no on sale today,\
      but remiond the customer to look for hats"

In [25]:
# relevant system messages is essentially a simpler concept of what RAGS are 
def chat(message, history):
    relevant_system_mesasge = system_message
    if 'belt' in message:
        relevant_system_mesasge += "the store does not sell belts; if you are asked for belts, be sure to point out other items on sale."

    messages = [{'role':'system', 'content': relevant_system_mesasge}] + history + [{'role':'user', 'content':message}]

    stream = ollama.chat.completions.create(model='llama3.1', messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [26]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.
